# 1. Importing Libraries

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import OrderedDict
import os
import math
import re
import io

# 2. Define Variable Initializations

In [31]:
# Set random seed
torch.manual_seed(1234)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1234)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
params = {
    "batch_size": 20,
    "seq_length": 20,
    "layers": 2,
    "decay": 2,
    "rnn_size": 200,
    "dropout": 0.0,
    "init_weight": 0.1,
    "lr": 1.0,
    "vocab_size": 10000,
    "max_epoch": 4,
    "max_max_epoch": 13,
    "max_grad_norm": 5,
}

# 3. Extract and Transform Data

In [32]:
ptb_path = "./data/"

vocab_idx = 0
vocab_map = {}

In [33]:
def tokenize(text):
    """ Mimic Lua's stringx.split() behavior. """
    return re.findall(r"\S+", text)  # Split by whitespace but keep everything else intact.

def load_data(fname):
    global vocab_idx
    
    print(f"\n[Loading Data] File: {fname}")
    
    # Read file and replace newlines with <eos>
    with open(fname, 'r', encoding='utf-8') as f:
        data = f.read()
    
    data = data.replace('\n', '<eos>')  # Match Lua behavior
    data = data.split()  # Equivalent to stringx.split in Lua
            
    x = torch.zeros(len(data), dtype=torch.long)
    
    for i in range(len(data)):
        if data[i] not in vocab_map:
            vocab_idx += 1
            vocab_map[data[i]] = vocab_idx
        x[i] = vocab_map[data[i]]
    return x

def replicate(x_inp, batch_size):
    """ Replicates and shifts data exactly like Lua """
    s = x_inp.size(0)
    x = torch.zeros((s // batch_size, batch_size), dtype=torch.long)

    for i in range(batch_size):
        start = round(i * s / batch_size)  # Mimic Lua rounding
        finish = start + x.size(0)
        x[:, i] = x_inp[start:finish]
    return x

def traindataset(batch_size):
    x = load_data(os.path.join(ptb_path, "ptb.train.txt"))
    x = replicate(x, batch_size)
    return x

def testdataset(batch_size):
    x = load_data(os.path.join(ptb_path, "ptb.test.txt"))
    x = x.view(-1, 1).expand(-1, batch_size).clone()
    return x

def validdataset(batch_size):
    x = load_data(os.path.join(ptb_path, "ptb.valid.txt"))
    x = replicate(x, batch_size)
    return x

datasets = {
    "traindataset": traindataset,
    "validdataset": validdataset,
    "testdataset": testdataset
}

In [34]:
# Call dataset functions

data_train = traindataset(params["batch_size"])
data_valid = validdataset(params["batch_size"])
data_test = testdataset(params["batch_size"])


[Loading Data] File: ./data/ptb.train.txt

[Loading Data] File: ./data/ptb.valid.txt

[Loading Data] File: ./data/ptb.test.txt


# 4. LSTM Cell Class 

In [35]:
# LSTM Cell implementation
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()
        self.W = nn.Linear(input_size, 4*hidden_size)
        self.U = nn.Linear(hidden_size, 4*hidden_size)
    
    def forward(self, x, prev_c, prev_h):
        gates = self.W(x) + self.U(prev_h)
        i, f, o, g = gates.chunk(4, dim=-1)
        i, f, o = torch.sigmoid(i), torch.sigmoid(f), torch.sigmoid(o)
        g = torch.tanh(g)
        next_c = f * prev_c + i * g
        next_h = o * torch.tanh(next_c)
        return next_c, next_h

# 5. LSTM Model

In [36]:
# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, params):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(params["vocab_size"], params["rnn_size"])
        self.lstm_cells = nn.ModuleList([
            LSTMCell(params["rnn_size"], params["rnn_size"]) for _ in range(params["layers"])
        ])
        self.fc = nn.Linear(params["rnn_size"], params["vocab_size"])
        self.dropout = nn.Dropout(params["dropout"])
        self.init_weights()

    def init_weights(self):
        for param in self.parameters():
            param.data.uniform_(-params["init_weight"], params["init_weight"])

    def forward(self, x, hidden_states):
        x = self.embedding(x)
        next_hidden_states = []
        for i, cell in enumerate(self.lstm_cells):
            prev_c, prev_h = hidden_states[i]
            next_c, next_h = cell(x, prev_c, prev_h)
            x = next_h  # Input for next layer
            next_hidden_states.append((next_c, next_h))
        x = self.dropout(x)
        x = self.fc(x)
        return torch.log_softmax(x, dim=-1), next_hidden_states

    def init_hidden(self, batch_size):
        return [(torch.zeros(batch_size, params["rnn_size"]).to(device),
                 torch.zeros(batch_size, params["rnn_size"]).to(device))
                for _ in range(params["layers"])]

In [37]:
# Initialize model
model = LSTMModel(params).to(device)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=params["lr"])

# 6. Train and Evaluate Model

In [38]:
def g_disable_dropout(node):
    if isinstance(node, list) and not hasattr(node, "__typename"):
        for i in range(len(node)):
            g_disable_dropout(node[i])
        return
    if hasattr(node, "__typename") and re.match("Dropout", node.__typename):
        node.train = False

def g_enable_dropout(node):
    if isinstance(node, list) and not hasattr(node, "__typename"):
        for i in range(len(node)):
            g_enable_dropout(node[i])
        return
    if hasattr(node, "__typename") and re.match("Dropout", node.__typename):
        node.train = True

def g_clone_many_times(net, T):
    clones = []
    params, grad_params = net.parameters()
    mem = io.BytesIO()
    torch.save(net, mem)
    for t in range(T):
        mem.seek(0)  # Reset buffer pointer like MemoryFile
        clone = torch.load(mem)
        clone_params, clone_grad_params = clone.parameters()
        for i in range(len(params)):
            clone_params[i].data.set_(params[i].data)
            clone_grad_params[i].data.set_(grad_params[i].data)
        clones.append(clone)
        torch.cuda.empty_cache()
    return clones

def g_init_gpu(args):
    gpuidx = args[0] if args else 1
    print("Using {}-th gpu".format(gpuidx))
    torch.cuda.set_device(gpuidx - 1)  # Match Lua 1-based indexing
    g_make_deterministic(1)

def g_make_deterministic(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.FloatTensor(1, 1).uniform_()

def g_replace_table(to, from_):
    assert len(to) == len(from_)
    for i in range(len(to)):
        to[i].copy_(from_[i])

def g_f3(f):
    return "{:.3f}".format(f)

def g_d(f):
    return "{}".format(int(round(f)))

In [ ]:
def fp(data, hidden_states):
    model.train()
    x = data[:-1]
    y = data[1:]
    x, y = x.to(device), y.to(device)
    output, hidden_states = model(x, hidden_states)
    loss = criterion(output.view(-1, params["vocab_size"]), y.view(-1))
    return loss, hidden_states

def bp(loss):
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), params["max_grad_norm"])
    optimizer.step()
    import gc
    gc.collect()

def run_epoch(data):
    hidden_states = model.init_hidden(params["batch_size"])
    total_loss = 0
    num_batches = max(1, data.size(0) // params["seq_length"] - 1)
    for batch in range(num_batches):
        x = data[batch * params["seq_length"]:(batch + 1) * params["seq_length"]]
        y = data[batch * params["seq_length"] + 1:(batch + 1) * params["seq_length"] + 1]
        loss, hidden_states = fp(x, hidden_states)
        bp(loss)
        total_loss += loss.item()
        hidden_states = [(c.detach(), h.detach()) for (c, h) in hidden_states]
    return torch.exp(torch.tensor(total_loss / num_batches))

def evaluate(data):
    model.eval()
    hidden_states = model.init_hidden(params["batch_size"])
    total_loss = 0
    num_batches = max(1, data.size(0) // params["seq_length"] - 1)
    with torch.no_grad():
        for batch in range(num_batches):
            x = data[batch * params["seq_length"]:(batch + 1) * params["seq_length"]]
            y = data[batch * params["seq_length"] + 1:(batch + 1) * params["seq_length"] + 1]
            loss, hidden_states = fp(x, hidden_states)
            total_loss += loss.item()
            hidden_states = [(c.detach(), h.detach()) for (c, h) in hidden_states]
    return torch.exp(torch.tensor(total_loss / num_batches))

In [40]:
# Training loop
for epoch in range(params["max_max_epoch"]):
    train_ppl = run_epoch(data_train)
    valid_ppl = evaluate(data_valid)
    print(f"Epoch {epoch+1}, Train PPL: {train_ppl:.2f}, Valid PPL: {valid_ppl:.2f}")
    if epoch > params["max_epoch"]:
        params["lr"] /= params["decay"]

# Test evaluation
test_ppl = evaluate(data_test)
print(f"Test PPL: {test_ppl:.2f}")

AssertionError: Torch not compiled with CUDA enabled